In [3]:
import sys
print(sys.version)

3.8.6 (default, Jan 27 2021, 15:42:20) 
[GCC 10.2.0]


In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
import pyvista as pv
from itkwidgets import view
%matplotlib widget
pv.rcParams['use_ipyvtk'] = True
pd.set_option('precision', 6)

In [3]:
#%cat 'data/synth.dat'

In [2]:
%pip freeze  > requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Read raw datas from tecplot

In [76]:
df = pd.read_csv(filepath_or_buffer=r'data/input_data/synth.dat',sep='\t',skiprows=10,
                 names=["X","Y","Z","Layer1","Layer2","Layer3","Layer4","Layer5"],dtype=np.float64)

In [77]:
layers=list(df.columns[3:]) # X,Y,Z
df.head()

,X,Y,Z,Layer1,Layer2,Layer3,Layer4,Layer5
0,500.0,500.0,0.5,1.0,0.0,0.0,0.0,0.0
1,500.0,500.0,1.5,1.0,0.0,0.0,0.0,0.0
2,500.0,500.0,2.5,1.0,0.0,0.0,0.0,0.0
3,500.0,500.0,3.5,1.0,0.0,0.0,0.0,0.0
4,500.0,500.0,4.5,1.0,0.0,0.0,0.0,0.0


### Function for filtering / sorting and merging layers from raw datas

In [78]:
def mask_df(df,formation):
    df_f = df.copy()
    df_f = df[df[formation]==1]
    l=layers.copy() 
    l.remove(formation)
    df_f=df_f.drop(columns=l)
    df_f.rename(columns={formation: 'formation'}, inplace=True)
    df_f=df_f.assign(formation=str(formation))
    return df_f

In [79]:
def sort_df(df,formation):
    df_l = mask_df(df,formation)
    groups = df_l.groupby(['X','Y'])
    df_sort = groups.max()
    df_sort.reset_index(inplace=True)
    return df_sort

In [80]:
def grad_df(df,normals,formation):
    df_grad = sort_df(df,formation)
    df_grad.insert(loc=3, column = 'G_x', value=normals[:,0])
    df_grad.insert(loc=4, column = 'G_y', value=normals[:,1])
    df_grad.insert(loc=5, column = 'G_z', value=normals[:,2])
    return df_grad

### Concatenate all layers point positions in a surface dataFrame

In [81]:
df_all_surf = pd.concat([sort_df(df,layer) for layer in layers],sort=True)
np.shape(df_all_surf)
df_all_surf.tail()

,X,Y,Z,formation
215,9500.0,7500.0,24.5,Layer5
216,9500.0,8000.0,24.5,Layer5
217,9500.0,8500.0,24.5,Layer5
218,9500.0,9000.0,24.5,Layer5
219,9500.0,9500.0,24.5,Layer5


### Compute Azimuth, dip angles from pole gradient vectors (normals definition)

In [82]:
def calculate_orientations(df, idx=None):
        if idx is None:
            pol = 1
            df.insert(loc=6,column = 'azimuth',value = np.rad2deg(np.nan_to_num(np.arctan2(df["G_x"] / pol,
                                                                                           df["G_y"] / pol))))
            df.insert(loc=7,column = 'dip', value = np.rad2deg(np.nan_to_num(np.arccos(df["G_z"] / pol))))
            df.insert(loc=8,column = 'polarity',value  = pol)
            # mask
            df["azimuth"][df["azimuth"] < 0] += 360  # shift values from [-pi, 0] to [pi,2*pi]
            df["azimuth"][df["dip"] < 0.001] = 0  # because if dip is zero azimuth is undefined
        else:
            df.loc[idx, 'polarity'] = 1
            df.loc[idx, "dip"] = np.rad2deg(np.nan_to_num(np.arccos(df.loc[idx, "G_z"] /
                                                                    df.loc[idx, "polarity"])))
            df.loc[idx, "azimuth"] = np.rad2deg(np.nan_to_num(
                np.arctan2(df.loc[idx, "G_x"] / df.loc[idx, "polarity"],
                           df.loc[idx, "G_y"] / df.loc[idx, "polarity"])))
            df["azimuth"][df["azimuth"] < 0] += 360  # shift values from [-pi, 0] to [pi,2*pi]
            df["azimuth"][df["dip"] < 0.001] = 0  # because if dip is zero azimuth is undefined
        return df

### Compute pole gradients ```G_x, G_y, G_z``` as normal vectors from a surface layer (```delaunay_2d```)

In [130]:
def plot_layers_orientation(ve,fact_n,is_l1,is_l2,is_l3,is_l4,is_l5):
    clr = ['b','orange','g','r','m']
    id=0
    is_layer = [is_l1,is_l2,is_l3,is_l4,is_l5]
    df_all_grad = pd.DataFrame(columns = ['X','Y','Z','G_x', 'G_y', 'G_z','formation'])
    # Set the coordinates from the numpy array
    p.remove_legend()
    p.clear()
    for layer in layers:
        if is_layer[layers.index(layer)] :
            df_sort = sort_df(df,layer)

            pts_x,pts_y,pts_z = [df_sort.X.to_numpy(),df_sort.Y.to_numpy(),df_sort.Z.to_numpy()]
            points = np.c_[pts_x, pts_y, ve*pts_z]
            print(np.shape(points))
            cloud = pv.PolyData(points)
            surf = cloud.delaunay_2d(tol=1e-5,progress_bar=True,offset=1e-5)
            surf.compute_normals(cell_normals=False, point_normals=True,inplace=True)  # this activates the normals as well
            normals = surf['Normals']
            df_all_grad=df_all_grad.append(grad_df(df,normals,layer),ignore_index=True)
            # plot 
            arrows = surf.glyph(scale="Normals", orient="Normals", tolerance=0.01,factor=fact_n)
            mesh.points = points
            glyph_act = p.add_mesh(arrows, color="c",point_size=10)
            surf_act = p.add_mesh(surf,color=clr[id],smooth_shading=True,label=layer,show_edges=True,opacity=1,lighting=True)
            id +=1
    p.show_grid()
    p.add_axes()
    p.add_legend()
    p.show() 

    df_all = calculate_orientations(df_all_grad) # insert azimuth, dip, polarity from normals calculation

    
mesh = pv.UnstructuredGrid()
p = pv.Plotter()

widgets.interact(plot_layers_orientation,
                 ve=widgets.IntSlider(description='Z vert. ex.',min=1,max=1000,step=10,value=200),
                 fact_n=widgets.IntSlider(description='Glyph ex.',min=1,max=1000,step=10,value=500),
                 is_l1=widgets.Checkbox(description='Layer 1',value=True),
                 is_l2=widgets.Checkbox(description='Layer 2',value=False),
                 is_l3=widgets.Checkbox(description='Layer 3',value=False),
                 is_l4=widgets.Checkbox(description='Layer 4',value=False),
                 is_l5=widgets.Checkbox(description='Layer 5',value=False)) 

interactive(children=(IntSlider(value=200, description='Z vert. ex.', max=1000, min=1, step=10), IntSlider(val…

<function __main__.plot_layers_orientation(ve, fact_n, is_l1, is_l2, is_l3, is_l4, is_l5)>

In [131]:
df_all_grad.head()

,X,Y,Z,G_x,G_y,G_z,azimuth,dip,polarity,formation
0,500.0,500.0,18.5,0.099015,0.099015,-0.990148,45.000000,171.950516,1,Layer1
1,500.0,1000.0,18.5,0.131876,0.065938,-0.989071,63.434948,171.521286,1,Layer1
2,500.0,1500.0,19.5,0.000000,0.196116,-0.980581,0.000000,168.690048,1,Layer1
3,500.0,2000.0,20.5,0.000000,0.065656,-0.997842,0.000000,176.235519,1,Layer1
4,500.0,2500.0,20.5,0.000000,0.131312,-0.991341,0.000000,172.454590,1,Layer1


In [132]:
df_all.mean()

X           5457.300275
Y           5178.719008
Z             19.838154
G_x           -0.081703
G_y           -0.010463
G_z           -0.673779
azimuth      163.908615
dip          143.227249
polarity       1.000000
dtype: float64

### And the Vasarely's plot

In [143]:
import matplotlib.pyplot as plt
def plot_elevation_map(layer):
    df_sort = sort_df(df,layer)
    pts_x,pts_y,pts_z = [df_sort.X.to_numpy(),df_sort.Y.to_numpy(),df_sort.Z.to_numpy()]
    points = np.c_[pts_x, pts_y, pts_z]

    plt.figure(figsize=(5, 5))
    plt.scatter(points[:, 0], points[:, 1], c=points[:, 2],s=100,marker="o",cmap="viridis")
    plt.axis("image")
    plt.colorbar()
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.show()
widgets.interact(plot_elevation_map,
                layer=widgets.Combobox(description='Layer',options=["Layer1", "Layer2", "Layer3", "Layer4", "Layer5"], value="Layer1"))

interactive(children=(Combobox(value='Layer1', description='Layer', options=('Layer1', 'Layer2', 'Layer3', 'La…

<function __main__.plot_elevation_map(layer)>

### Save surface dataFrame in ```csv``` format

In [144]:
def save_surface_in_csv(nbin,rand,is_l1,is_l2,is_l3,is_l4,is_l5):
    df_surf_mask = pd.DataFrame(columns = ['X','Y','Z','formation'])
    df_surf = df_all_surf.sample(n=nbin,random_state=rand) # bins
    df_surf['Z']=df_surf['Z']*100 # vertical exageration
    if is_l1:
        df_surf_mask = df_surf_mask.append(df_surf[df_surf['formation']=='Layer1'])
    if is_l2:
        df_surf_mask = df_surf_mask.append(df_surf[df_surf['formation']=='Layer2'])
    if is_l3:
        df_surf_mask = df_surf_mask.append(df_surf[df_surf['formation']=='Layer3'])
    if is_l4:
        df_surf_mask = df_surf_mask.append(df_surf[df_surf['formation']=='Layer4'])
    if is_l5:
        df_surf_mask = df_surf_mask.append(df_surf[df_surf['formation']=='Layer5'])
    df_surf_mask.to_csv(r'data/df_synth_surf.csv', sep=',', encoding='utf-8',index=False)
    print(df_surf_mask.head())
    print('... surface file saved')
    
widgets.interact(save_surface_in_csv,
                 nbin=widgets.IntSlider(description='Number of bins',min=10,max=len(sort_df(df,"Layer1")),step=10,value=50),
                 rand=widgets.IntSlider(description='random state number',min=1,max=10,step=1,value=2),
                 is_l1=widgets.Checkbox(description='Layer 1',value=True),
                 is_l2=widgets.Checkbox(description='Layer 2',value=True),
                 is_l3=widgets.Checkbox(description='Layer 3',value=True),
                 is_l4=widgets.Checkbox(description='Layer 4',value=True),
                 is_l5=widgets.Checkbox(description='Layer 5',value=False))

interactive(children=(IntSlider(value=50, description='Number of bins', max=361, min=10, step=10), IntSlider(v…

<function __main__.save_surface_in_csv(nbin, rand, is_l1, is_l2, is_l3, is_l4, is_l5)>

### Save orientation dataFrame in ```csv``` format

In [145]:
def save_orientation_in_csv(pole_gradient,azimuth_angle,nbin,rand,is_l1,is_l2,is_l3,is_l4,is_l5):
    df_grad_mask = pd.DataFrame(columns = ['X','Y','Z','G_x', 'G_y', 'G_z','azimuth', 'dip', 'polarity','formation'])
    df_grad = df_all_grad.sample(n=nbin,random_state=rand) # bins
    if (not pole_gradient):
        df_grad.drop(columns = ['G_x', 'G_y', 'G_z'],inplace = True)
        df_grad_mask.drop(columns = ['G_x', 'G_y', 'G_z'],inplace = True)
    if (not azimuth_angle):
        df_grad.drop(columns = ['azimuth', 'dip','polarity'],inplace = True)
        df_grad_mask.drop(columns = ['azimuth', 'dip','polarity'],inplace = True)
    df_grad['Z']=df_grad['Z']*100 # vertical exageration
    if is_l1:
        df_grad_mask = df_grad_mask.append(df_grad[df_grad['formation']=='Layer1'])
    if is_l2:
        df_grad_mask = df_grad_mask.append(df_grad[df_grad['formation']=='Layer2'])
    if is_l3:
        df_grad_mask = df_grad_mask.append(df_grad[df_grad['formation']=='Layer3'])
    if is_l4:
        df_grad_mask = df_grad_mask.append(df_grad[df_grad['formation']=='Layer4'])
    if is_l5:
        df_grad_mask = df_grad_mask.append(df_grad[df_grad['formation']=='Layer5'])
    df_grad_mask.to_csv(r'data/df_synth_orient.csv', sep=',', encoding='utf-8',index=False)
    print(df_grad_mask.head())
    print('... orientations file saved')
    
widgets.interact(save_orientation_in_csv,
                 pole_gradient=widgets.Checkbox(description='Gradient',value=True),
                 azimuth_angle=widgets.Checkbox(description='Azimuth',value=True),
                 nbin=widgets.IntSlider(description='Number of bins',min=10,max=len(sort_df(df,"Layer1")),step=10,value=50),
                 rand=widgets.IntSlider(description='random state number',min=1,max=10,step=1,value=2),
                 is_l1=widgets.Checkbox(description='Layer 1',value=True),
                 is_l2=widgets.Checkbox(description='Layer 2',value=True),
                 is_l3=widgets.Checkbox(description='Layer 3',value=True),
                 is_l4=widgets.Checkbox(description='Layer 4',value=True),
                 is_l5=widgets.Checkbox(description='Layer 5',value=False))

interactive(children=(Checkbox(value=True, description='Gradient'), Checkbox(value=True, description='Azimuth'…

<function __main__.save_orientation_in_csv(pole_gradient, azimuth_angle, nbin, rand, is_l1, is_l2, is_l3, is_l4, is_l5)>

In [146]:
sns.set(style = "darkgrid")
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
for layer in layers :
    df_l = mask_df(df,layer)
    x,y,z = [df_l['X'], df_l['Y'], df_l['Z']]
    ax.scatter(x, y, z,s=30,depthshade=True)
plt.grid(b=True,axis='both')
plt.show(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Use GemPy and theano interpolation

In [149]:
import gempy as gp
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu"

In [159]:
res_x,res_y,res_z = [50,50,100]
geo_data = gp.create_data('synth_model_dimitri',
                           extent=[0, 10000, 0, 10000, 0, 2500],
                           resolution=[res_x,res_y,res_z],
                           path_i ="data/df_synth_surf.csv",
                           path_o ="data/df_synth_orient.csv",default_value=True)
#gp.set_orientation_from_surface_points(geo_data,[0,1,2])
gp.get_data(geo_data,'orientations').describe()

Active grids: ['regular']


If pole_vector and orientation are passed pole_vector is used/


,X,Y,Z,G_x,G_y,G_z,smooth
count,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,4.600000e+01
mean,5076.086957,5945.652174,1923.913043,-0.072779,-0.056969,-0.977639,1.000000e-02
std,2579.766121,2461.319115,475.831335,0.124226,0.143189,0.023163,1.753892e-18
min,500.000000,500.000000,550.000000,-0.410415,-0.357199,-1.000000,1.000000e-02
25%,3000.000000,4250.000000,1575.000000,-0.151132,-0.146246,-0.994253,1.000000e-02
50%,4750.000000,6500.000000,2100.000000,-0.074291,-0.057895,-0.985307,1.000000e-02
75%,7375.000000,7500.000000,2250.000000,0.021163,0.044315,-0.964729,1.000000e-02
max,9000.000000,9500.000000,2450.000000,0.130628,0.212718,-0.907746,1.000000e-02


In [ ]:
#layer_names = ['Layer1']
#for lay in layer_names:
#    lay_idx = geo_data.surface_points.df.index[geo_data.surface_points.df['surface'] == lay]
#    gp.set_orientation_from_surface_points(geo_data, lay_idx)
#    print(len(lay_idx))

### Load a random topography

In [160]:
geo_data.surfaces
geo_data.set_topography(source='random')

[2000. 2500.]
Active grids: ['regular' 'topography']


Grid Object. Values: 
array([[  100.        ,   100.        ,    12.5       ],
       [  100.        ,   100.        ,    37.5       ],
       [  100.        ,   100.        ,    62.5       ],
       ...,
       [10000.        ,  9591.83673469,  2329.85357324],
       [10000.        ,  9795.91836735,  2325.39877744],
       [10000.        , 10000.        ,  2313.27573895]])

### Declare series and surface order

In [161]:
gp.map_stack_to_surfaces(geo_data, {"Strat_Series": ("Layer4","Layer3","Layer2","Layer1"),
                                    "Basement Series":"basement"},
                                   remove_unused_series=False)
gp.map_series_to_surfaces(geo_data, {"Strat_series":('basement','Layer4','Layer3','Layer2','Layer1')},remove_unused_series=False)
geo_data.surfaces.colors.change_colors({'Layer1': '#015482', 'Layer2': '#e5350f', 'Layer3': '#728f02','Layer4': '#9f0052',
                                        'basement': '#443988'})
geo_data.surfaces

,surface,series,order_surfaces,color,id
0,Layer1,Strat_series,1,#015482,1
1,Layer2,Strat_series,2,#e5350f,2
2,Layer3,Strat_series,3,#728f02,3
3,Layer4,Strat_series,4,#9f0052,4
4,basement,Strat_series,5,#443988,5


In [257]:
#order_series = ["Strat_Series_3","Strat_Series_2","Strat_Series_1","Basement_Series"]
#geo_data.reorder_series(order_series)

In [162]:
interp_data = gp.set_interpolator(geo_data, compile_theano=True,theano_optimizer='fast_compile')

Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                           values
range                    14361.4
$C_o$                4.91071e+06
drift equations  [3, 3, 3, 3, 3]


In [ ]:
#new_range = geo_data.get_additional_data().loc[('Kriging', 'range'), 'values'] * 0.2
#geo_data.modify_kriging_parameters('range', new_range)

In [ ]:
#geo_data.orientation
#geo_data.grid
#gp.get_additional_data(geo_data)

In [163]:
sol = gp.compute_model(geo_data,set_solutions=True, compute_mesh=True)

In [164]:
sol.lith_block

array([1., 1., 1., ..., 5., 5., 5.])

In [165]:
sol.grid

Grid Object. Values: 
array([[  100.        ,   100.        ,    12.5       ],
       [  100.        ,   100.        ,    37.5       ],
       [  100.        ,   100.        ,    62.5       ],
       ...,
       [10000.        ,  9591.83673469,  2329.85357324],
       [10000.        ,  9795.91836735,  2325.39877744],
       [10000.        , 10000.        ,  2313.27573895]])

In [166]:
def plot_cross_section(cell,layer,dir):
    gp.plot_2d(geo_data, cell_number=cell, series_n=layer,direction=dir,ve=2,
               show_data=True,
               show_boundaries=True,
               show_results=True,
               show_topography=True,kwargs_regular_grid={'cmap': 'viridis', 'norm':None})
    #plt.show()
widgets.interact(plot_cross_section,
                 cell=widgets.IntSlider(description='slice on x/y :',min=0,max=res_x-1,step=1,value=10),
                 layer=[('Layer1', 0), ('Layer2', 1), ('Layer3', 2), ('Layer4', 3), ('Layer5', 4)],
                 dir=['x','y'])

interactive(children=(IntSlider(value=10, description='slice on x/y :', max=49), Dropdown(description='layer',…

<function __main__.plot_cross_section(cell, layer, dir)>

In [183]:
gp.plot_3d(geo_data, plotter_type='background',ve=1,show_topography=True,show_data=True,show_surfaces=True,show_results=True,show_lith=True)

In [168]:
#gp.plot.plot_interactive_3d(geo_data)

### Save model

In [178]:
# Simple export to VTK:
def export_to_vtk(sol,filename):
    import pyevtk
    print('regular grid:\t', sol.grid.regular_grid.resolution, '\t', sol.grid.regular_grid.extent.astype(int))

    #Export whole, uncropped lith_block
    #Get coordinate info from grid & create VTK cells info:
    xmin = sol.grid.regular_grid.extent[0]
    xmax = sol.grid.regular_grid.extent[1]
    xres = sol.grid.regular_grid.resolution[0]
    dx   = (xmax-xmin)/xres                       #pixel width
    xvals = np.arange(xmin,xmax+dx,dx)

    ymin = sol.grid.regular_grid.extent[2]
    ymax = sol.grid.regular_grid.extent[3]
    yres = sol.grid.regular_grid.resolution[1]
    dy   = (ymax-ymin)/yres
    yvals = np.arange(ymin,ymax+dy,dy)

    zmin = sol.grid.regular_grid.extent[4]
    zmax = sol.grid.regular_grid.extent[5]
    zres = sol.grid.regular_grid.resolution[2]
    dz   = (zmax-zmin)/zres
    zvals = np.arange(zmin,zmax+dz,dz)

    print('x:', xmin,xmax,xres,dx)
    print('y:', ymin,ymax,yres,dy)
    print('z:', zmin,zmax,zres,dz)

    g = sol.lith_block.copy()            #make a copy to avoid messing up original
    g = np.reshape(g, (xres,yres,zres))  #reshape lith block to 3D
    print('shape of array to export:', g.shape)
    plt.imshow(g[:,:,zres-1])
    path = r'data/'+str(filename)  #set file path to save to (should have no extension)
    pyevtk.hl.gridToVTK(path, xvals, yvals, zvals, cellData={'data': g}) #export to VTK

In [179]:
export_to_vtk(sol,'test')

regular grid:	 [ 50  50 100] 	 [    0 10000     0 10000     0  2500]
x: 0.0 10000.0 50 200.0
y: 0.0 10000.0 50 200.0
z: 0.0 2500.0 100 25.0
shape of array to export: (50, 50, 100)


In [170]:
gp.save_model(geo_data)
#np.save('Layer1_ver', geo_data.solutions.vertices) # numpy binary export .npy
#np.save('Layer1_edges', geo_data.solutions.edges)